In [1]:
from time import sleep

import sys

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import json
import pandas as pd
import time
import requests
import concurrent.futures

from PIL import Image

In [ ]:
options = webdriver.chrome.options.Options()
options.add_argument('--headless')
options.add_argument('--no-sandbox')

path_driver = '/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/chromedriver'
driver = webdriver.Chrome(path_driver, options = options)
driver.get('https://wego.here.com/directions/mix//tycho-brahes-alle-21-st-th-2300-koebenhavn-s-01017740__21_st__th')
sleep(5)
driver.get_screenshot_as_file("screenshot.png")

In [ ]:
def get_screenshots_maps(house_from, house_till):
    options = webdriver.chrome.options.Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')

    path_driver = '/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/chromedriver'
    driver = webdriver.Chrome(path_driver, options = options)

    left, upper, right, lower = 800, 200, 1550, 800 #for picture cropping


    with concurrent.futures.ThreadPoolExecutor() as executor:
        threaded_start = time.time()
        for x in tqdm(range (house_from, house_till)):
    
            path = '/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/screenshots/'\
                'here_wego_screenshot_' + str(x) + '.png'
            try:
                driver.get('https://wego.here.com/directions/mix//' \
                           + df_Boligsiden1["address.latestForSale.addressId"][x])
                sleep(3)
                driver.get_screenshot_as_file(path)
        
                im = Image.open(path)

                im_crop = im.crop((left, upper, right, lower))
                im_crop.save('/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/screenshots/'\
                             'here_wego_screenshot.png', quality=95)

            except:
                print('Could not find address', df_Boligsiden1["address.latestForSale.addressId"][x])
                pass
        print("threaded time:", time.time() - threaded_start)
    return


In [ ]:
def get_screenshot_maps(address):
    options = webdriver.chrome.options.Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')

    path_driver = '/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/chromedriver'
    driver = webdriver.Chrome(path_driver, options = options)

    path = '/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/screenshots/'\
                'here_wego_screenshot.png'
    
    driver.get('https://wego.here.com/directions/mix//' \
                + address)
    sleep(5)
    driver.get_screenshot_as_file(path)
        
    im = Image.open(path)
    
    left, upper, right, lower = 925, 375, 1425, 875 #for picture cropping
    im_crop = im.crop((left, upper, right, lower))
    
    im_crop.save('/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/screenshots/'\
                             'here_wego_screenshot.png', quality=95)
    
    
    return im_crop

In [ ]:
def crop_img(get_address):
    path = '/Users/thommasbiener/Desktop/MAT-ØK.KU/Speciale/programming/screenshots/'\
                'here_wego_screenshot.png'

    im = cv.imread(path)
    im = cv.cvtColor(im, cv.COLOR_BGR2RGB)
    
    dim = (1000, 1000)
    # resize image
    im = cv.resize(im, dim, interpolation = cv.INTER_AREA)

    # Make new image to set as background
    lum_img = Image.new(mode = "RGB", size = (1000, 1000),
                           color = (255, 255, 255))

    draw = ImageDraw.Draw(lum_img)
    draw.pieslice([(0,0), (1000, 1000)], 0 , 360,
                  fill = 0, outline = "white")

    img_arr = np.array(im)
    lum_img_arr = np.array(lum_img)

    # Switch where color [0, 0, 0] is not to the picture
    img = np.where(lum_img_arr == [0, 0, 0], img_arr, 255)
    return img

In [ ]:
def get_percent_color(color, maps_img):
    mylist = [-4, -3, -2, -1, 0 , 1, 2, 3, 4]
    
    R_color = list(map(lambda x: x + color[0], mylist))
    G_color = list(map(lambda x: x + color[1], mylist))
    B_color = list(map(lambda x: x + color[2], mylist))
    
    K = np.where((maps_img[...,0] == R_color[0]) | (maps_img[...,0] == R_color[1]) | 
                 (maps_img[...,0] == R_color[2]) | (maps_img[...,0] == R_color[3]) | 
                 (maps_img[...,0] == R_color[4]) | (maps_img[...,0] == R_color[5]) |
                 (maps_img[...,0] == R_color[6]) | (maps_img[...,0] == R_color[7]) | 
                 (maps_img[...,0] == R_color[7]),
                 maps_img[...,1], 0)
    
    K = np.where((K == G_color[0]) | (K == G_color[1]) | (K == G_color[2]) |
                 (K == G_color[3]) | (K == G_color[4]) | (K == G_color[5]) |
                 (K == G_color[6]) | (K == G_color[7]) | (K == G_color[7]), 
                 img_2[...,2], 0)
    
    K = np.where((K == B_color[0]) | (K == B_color[1]) | (K == B_color[2]) |
                 (K == B_color[3]) | (K == B_color[4]) | (K == B_color[5]) |
                 (K == B_color[6]) | (K == B_color[7]) | (K == B_color[7]), 1, 0).sum()
    K = K/1000000
    return K

In [ ]:
color_list[["highway_red",
              "LargeRoad_salmon", 
              "BiggerRoad_Yellow",
              "BigRoad_Yellow",
              "water_blue",
              "buildings_grey",
              "Buildings_darkgrey",
              "Buildings_lightgrey", 
              "largePark_green",
              "smallerPark_green",
              "text_black",
              "street_white" ]]

In [ ]:
def get_hereWeGo(df):
    color_dict = {"highway_red": [240, 144, 178],
                  "LargeRoad_salmon": [255, 173, 157], 
                  "BiggerRoad_Yellow": [254, 251, 167],
                  "BigRoad_Yellow": [254, 254, 226],
                  "water_blue": [153, 206, 254],
                  "buildings_grey": [255, 252, 169],
                  "Buildings_darkgrey": [209, 218, 220],
                  "Buildings_lightgrey": [224, 230, 234], 
                  "largePark_green": [182, 231, 161],
                  "smallerPark_green": [212, 230, 218],
                  "text_black": [50, 50, 50],
                  "street_white": [255, 255, 255]}
        
    color_list = df_Boligsiden1
    address = color_list["address.latestForSale.addressId"][10]
    color_list["highway_red"] = color_list["address.latestForSale.addressId"]
    color_list["LargeRoad_salmon"] = color_list["address.latestForSale.addressId"]
    color_list["BiggerRoad_Yellow"] = color_list["address.latestForSale.addressId"]
    color_list["BigRoad_Yellow"] = color_list["address.latestForSale.addressId"]
    color_list["water_blue"] = color_list["address.latestForSale.addressId"]
    color_list["buildings_grey"] = color_list["address.latestForSale.addressId"]
    color_list["Buildings_darkgrey"] = color_list["address.latestForSale.addressId"]
    color_list["Buildings_lightgrey"] = color_list["address.latestForSale.addressId"]
    color_list["largePark_green"] = color_list["address.latestForSale.addressId"]
    color_list["smallerPark_green"] = color_list["address.latestForSale.addressId"]
    color_list["text_black"] = color_list["address.latestForSale.addressId"]
    color_list["street_white"] = color_list["address.latestForSale.addressId"]
    
    for row in tqdm(range(len(color_list[:10]))):
        img = crop_img(get_screenshot_maps(address[row]))
        sleep(2)
        for col in color_dict:
            color_list[str(col)][row] = get_percent_color(color_dict[col], img)
        
    return color_list

In [ ]:
get_hereWeGo(df_Boligsiden1["address.latestForSale.addressId"][:10])